### 문서 군집화
- 군집분석은 비지도 학습으로 비슷한 데이터들끼리 서로 군집을 이루는 것을 말한다. 이러한 방법은 텍스트로 이루어진 문서 데이터에도 적용이 된다. 
- 단어 발생 빈도수에 기반하는 BOW(Bag Of Words) 방식을 이용해 Feature(문서들을 이루고 있는 단어들)를 벡터화시키거나 단어들간의 의미 관계 즉, 단어 벡터들간의 방향을 고려해 Word embedding을 통해 벡터화 시킨다. 

#####  K-means를 활용한 Document Clustering
- 여기서는 BOW에 기반한 Tf-idf 방법을 사용한다. 다른 BOW 방식인 Count Vectorizer는 단순히 그저 단어 발생 빈도수에만 초점을 맞추기 때문에 여러가지 문서들간의 관계를 잘 고려하지 못한다. 

In [2]:
import pandas as pd
import glob, os

# 이스케이프 기능을 무시하고 그대로 문자열로 출력하고 싶을 때가 생긴다. 
# 그럴 때는 문자열 앞에 r을 붙이면 문자열 안에서 백 슬래시를 이스케이프 문자로 해석하지 않고 형태 그대로 출력
path = r'C:\cakd5\workspace\m8_머신러닝적용\dataset\OpinosisDataset1.0\topics'

# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list로 취합
# 개별 파일들의 파일내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text list로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경, 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    
    # 빈 그릇 만들어놓은 filne_list에 append함수 이용해 파일명과 파일 내용 뿌려줌
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [6]:
import pandas as pd
# 보기 너비 증가
pd.set_option('display.max.colwidth', 500)
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that t..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 ..."
2,battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book..."
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 ...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 ..."


In [16]:
document_df.loc[0,'opinion_text']

"                                                                                                                                                                                                                                                               , and is very, very accurate .\n0                                                                                                                                                                           but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1                                                                                                              This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .\n2                                                                                                                                         

### 데이터프레임 만들어놓고 벡터화 해주기 전에 데이터 전처리 작업
- Lemmatization (문법적 또는 의미적으로 변하는 단어의 원형을 찾는 방법)

##### 각 opinion_text에 들어있는 여러 문장들을 토큰화시키고 동시에 어근을 추출해주기 위해 NLTK의 Lemmatize 함수를 사용해보자.

In [5]:
# 텍스트 단어들의 어근 원형을 추출하기 위해 함수 생성
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string
import warnings
warnings.filterwarnings('ignore')
# string.puncutaion에 문자열의 모든 구두점이 들어있음
# 이를 활용해서 Tokenize시킬 때 구두점들을 제외하기 위한 것
# ord('문자열') => 문자열의 ASCII코드를 반환해줌!
# dict(key, value)형태로 모든 구두점의 각 ASCII코드를 key값으로 넣어주자!
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]
# 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
def LemNormalize(text):
    # .translate인자에 구두점 dict넣어주어서 구두점 삭제해준 상태로 토큰화시키기!
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Tf-idf 벡터화시키면서 cusotmized해준 토큰화+어근추출 방식 tokenizer인자에 넣어주기
# 벡터화시킬 Tf-idf 도구 옵션 추가해서 구축
# 1,2gram적용, 빈도수 0.05이하, 0.85이상의 빈도수 단어들 제거
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            stop_words='english', ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)
# fit_transform으로 위에서 구축한 도구로 텍스트 벡터화
ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

##### 각 문서들을 토큰화시키면서 어근을 추출했고 Tf-idf에 기반하여 단어들을 벡터화시켜 Feature들로 만드는 과정까지 마쳤다. 이제 K-means 알고리즘에 학습시켜 문서끼리 군집화를 시켜보자.

In [6]:
# K-means로 3개 군집으로 문서 군집화시키기
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=42)
# 비지도 학습이니 feature로만 학습시키고 예측
cluster_label = kmeans.fit_predict(ftr_vect)

# 군집화한 레이블값들을 document_df 에 추가하기
document_df['cluster_label'] = cluster_label
print(document_df.sort_values(by=['cluster_label']))

                           filename  \
25    performance_honda_accord_2008   
29        quality_toyota_camry_2007   
37          seats_honda_accord_2008   
22        mileage_honda_accord_2008   
18       interior_toyota_camry_2007   
17       interior_honda_accord_2008   
7         comfort_toyota_camry_2007   
16    gas_mileage_toyota_camry_2007   
47   transmission_toyota_camry_2007   
6         comfort_honda_accord_2008   
20   location_bestwestern_hotel_sfo   
24    parking_bestwestern_hotel_sfo   
38    service_bestwestern_hotel_sfo   
21      location_holiday_inn_london   
39       service_holiday_inn_london   
46          staff_swissotel_chicago   
28         price_holiday_inn_london   
45      staff_bestwestern_hotel_sfo   
15       free_bestwestern_hotel_sfo   
14           food_swissotel_chicago   
13          food_holiday_inn_london   
40  service_swissotel_hotel_chicago   
1    bathroom_bestwestern_hotel_sfo   
30      rooms_bestwestern_hotel_sfo   
31          rooms_swissot

In [18]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that t...",2
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 ...",0
2,battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book...",1
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 ...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 ...",1
